In [1]:
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

# 

Autosaving every 180 seconds


In [2]:
##############################################
################# IMPORTS ####################
##############################################
import numpy as np
import os

#
import pandas as pd
import h5py
from tifffile import tifffile, imread
import pathlib
import suite2p


###############################################
################ HELPER FUNCTIONS #############
###############################################

def get_mouse_ids(header,
                  dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='mouse ID')[0][0]

    ids = np.vstack(df.iloc[:,idx])

    return ids.squeeze()

def get_file_paths(header,
                  dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='path')[0][0]

    paths = np.vstack(df.iloc[:,idx])

    return paths.squeeze()

def get_sessions(header,
                 dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='sessions')[0][0]

    sessions = np.vstack(df.iloc[:,idx][:])

    return sessions.squeeze()


def get_comments(header,
                 dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='comments')[0][0]

    comments = np.vstack(df.iloc[:,idx][:])

    return comments.squeeze()


def convert_mesc_sessions_to_concatenated_tiff(fname, 
                                               sessions_in):
    
    fname_out = fname.replace('.mesc','.tif')

    if os.path.exists(fname_out):
        return fname_out
    
    sessions_in = sessions_in.replace(" ", "").split (",")

    #
    sess_list = [] 
    for session in sessions_in:
        temp = session.replace("S",'')
        temp = 'MUnit_'+str(int(temp)-1)
        print ("session loaded: ", temp)
        sess_list.append(temp)
    #
    data = []
    with h5py.File(fname, 'r') as file:
                #
        for sess in sess_list:
            print ("processing: ", sess)
            temp = file['MSession_0'][sess]['Channel_0'][()]
            print ("    data loaded size: ", temp.shape)
            data.append(temp)

    data = np.vstack(data)
    print(data.shape)

    # from tifffile import tifffile
    
    tifffile.imwrite(fname_out, data[:1000])
    
    return fname_out


def run_suite2p_from_fname(fname):

    data = imread(fname)
    print('imaging data of shape: ', data.shape)
    n_time, Ly, Lx = data.shape

    #
    ops = suite2p.default_ops()
    ops['batch_size'] = 200 # we will decrease the batch_size in case low RAM on computer
    ops['threshold_scaling'] = 2.0 # we are increasing the threshold for finding ROIs to limit the number of non-cell ROIs found (sometimes useful in gcamp injections)
    ops['fs'] = 30 # sampling rate of recording, determines binning for cell detection
    ops['tau'] = 1.25 # timescale of gcamp to use for deconvolution
    ops['save_path0'] = os.path.split(fname)[0]
    ops['save_folder'] = os.path.split(fname)[0]
    ops['tiff_list'] = [fname]

    print(ops)

    #
    db = {
        'data_path': os.path.split(fname)[0],
    }
    print(db)

    #
    output_ops = suite2p.run_s2p(ops=ops, db=db)

      

/home/cat/anaconda3/envs/suite2p/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally


In [3]:
################################################################################
############################## READ SPREADSHEET ################################
################################################################################
# #
# fname = 'smb://unibasel.ads.unibas.ch/bz/RG%20Donato/Microscopy/Mariona/DON10775/22-08-10/data/Image_001_001.raw'
# data = np.memmap(fname,
#                    dtype='uint16',
#                    mode='r')
# data = data.reshape(-1, 512*512)
# print (data.shape)


In [4]:
###############################################
################ GET LABELS ###################
###############################################
#

fname_xls = '/media/cat/4TB1/donato/steffen/scicore/scicore_suite2p_db_sk.xlsx'

#
df = pd.read_excel(fname_xls)
header = df.columns.tolist()

    
mouse_ids = get_mouse_ids(header, df)
print ("mouse ids: ", mouse_ids)

#
paths = get_file_paths(header, df)
print ("paths: ", paths, paths[0])

#
sessions = get_sessions(header, df)
print ("sessions: ", sessions)

#
comments = get_comments(header, df)
print ("comments: ", comments)

#
df.iloc[1, df.columns.get_loc('comments')] = 1

#
df.to_excel(fname_xls)  


mouse ids:  ['DON-009191' 'DON-009192' 'DON-009471' 'DON-009469']
paths:  ['\\Experiments\\DON-009191\\20220219\\002P-F\\DON-009191_20220219_002P-F_S1-S2.mesc'
 '\\Experiments\\DON-009192\\20220219\\002P-F\\DON-009192_20220219_002P-F_S1-S2.mesc'
 '\\Experiments\\DON-009471\\20220421\\002P-F\\DON-009471_20220421_002P-F_S1-S2-S3-S4-ACQ.mesc'
 '\\Experiments\\DON-009469\\20220420\\002P-F\\DON-009469_20220420_002P-F_S1-S2-ACQ.mesc'] \Experiments\DON-009191\20220219\002P-F\DON-009191_20220219_002P-F_S1-S2.mesc
sessions:  ['S1, S2, S3' 'S1, S2, S3' 'S3, S4' 'S1, S2']
comments:  [1 1 1 1]


In [27]:
#
#print (df['comments'])

for d_ in df['comments']:
    print (d_, d_==1.0)


1 True
1 True
1 True
1 True


In [5]:
##########################################################
################# CONVERT DATA TO .MSEC ##################
##########################################################

#
for k in range(len(paths)):
    #recording_id = 1

    fname = os.path.join(root_dir,
                         paths[k].replace(r'\\', '/'))
    p = pathlib.PureWindowsPath(fname)
    fname = (p.as_posix())

    #
    sessions_in = sessions[k]
    fname_tiff = convert_mesc_sessions_to_concatenated_tiff(fname,
                                                            sessions_in)
    
    #    
    run_suite2p_from_fname(fname_tiff)
                    

session loaded:  MUnit_0
session loaded:  MUnit_1
processing:  MUnit_0
    data loaded size:  (55748, 512, 512)
processing:  MUnit_1
    data loaded size:  (27874, 512, 512)
(83622, 512, 512)
imaging data of shape:  (1000, 512, 512)
{'suite2p_version': '0.10.3', 'look_one_level_down': False, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'nwb_file': '', 'nwb_driver': '', 'nwb_series': '', 'save_path0': '/media/cat/4TB1/donato/steffen/scicore/Steffen/Experiments/DON-009469/20220420/002P-F', 'save_folder': '/media/cat/4TB1/donato/steffen/scicore/Steffen/Experiments/DON-009469/20220420/002P-F', 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.25, 'fs': 30, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': False, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': False, 'save_NWB': False, 'combined': True, 'aspect': 1.0, 'do_bidipha

/home/cat/anaconda3/envs/suite2p/lib/python3.8/site-packages/suite2p/registration/utils.py:234: RuntimeWarning: invalid value encountered in apply_dotnorm
  return ifft2(apply_dotnorm(fft2(mov), img)) #.astype(np.complex64)


Reference frame, 16.93 sec.
Registered 800/1000 in 25.70s
added enhanced mean image
----------- Total 58.14 sec
NOTE: applying default /home/cat/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 38
Binned movie [26,502,498] in 1.01 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 1.00, threshold 20.00 
0 ROIs, score=51.10
Detected 4 ROIs, 0.47 sec
After removing overlaps, 4 ROIs remain
----------- Total 1.51 sec.
----------- EXTRACTION
Masks created, 0.64 sec.
Extracted fluorescence from 4 ROIs in 1000 frames, 5.77 sec.
----------- Total 6.43 sec.
----------- CLASSIFICATION
['npix_norm', 'skew', 'compact']
----------- Total 0.04 sec.
----------- SPIKE DECONVOLUTION
----------- Total 0.00 sec.
Plane 0 processed in 66.13 sec (can open in GUI).
total = 68.14 sec.
TOTAL RUNTIME 68.14 sec
session loaded:  MUnit_2
session loaded:  MUnit_3
processing:  MUnit_2
    data loaded size:  (37165, 512, 512)
processing:  MUnit_3
    data lo

/home/cat/anaconda3/envs/suite2p/lib/python3.8/site-packages/suite2p/registration/utils.py:234: RuntimeWarning: invalid value encountered in apply_dotnorm
  return ifft2(apply_dotnorm(fft2(mov), img)) #.astype(np.complex64)


Reference frame, 18.10 sec.
Registered 800/1000 in 25.91s
added enhanced mean image
----------- Total 59.59 sec
NOTE: applying default /home/cat/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 38
Binned movie [26,498,496] in 1.09 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 1.00, threshold 20.00 
0 ROIs, score=48.25
Detected 25 ROIs, 0.95 sec
After removing overlaps, 25 ROIs remain
----------- Total 2.18 sec.
----------- EXTRACTION
Masks created, 1.04 sec.
Extracted fluorescence from 25 ROIs in 1000 frames, 0.79 sec.
----------- Total 1.86 sec.
----------- CLASSIFICATION
['npix_norm', 'skew', 'compact']
----------- Total 0.03 sec.
----------- SPIKE DECONVOLUTION
----------- Total 0.00 sec.
Plane 0 processed in 63.67 sec (can open in GUI).
total = 66.08 sec.
TOTAL RUNTIME 66.08 sec


In [7]:
df = pd.read_excel(fname_xls)
header = df.columns.tolist()

print ("df: ", df)
print (len(df))

df:     Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Unnamed: 0.1.1.1  \
0           0             0               0                 0   
1           1             1               1                 1   
2           2             2               2                 2   
3           3             3               3                 3   

  Unnamed: 0.1.1.1.1      date    mouse ID sex   P pipeline  ...  size [um]  \
0                 2p  20220219  DON-009191   m  19  suite2p  ...    300x300   
1                 2p  20220219  DON-009192   m  19  suite2p  ...    300x300   
2                 2p  20220421  DON-009471   m  66  suite2p  ...    600x600   
3                 2p  20220420  DON-009469   m  65  suite2p  ...    600x600   

   n planes  z-stack vol. [um] fps [Hz]  frames dropped  binning [MxN]  \
0         1                  0       30               0              1   
1         1                  0       30               0              1   
2         1                  0       30            

In [12]:
# read pandas dataframe and process it
df = pd.read_excel(fname_xls)
header = df.columns.tolist()

# find # of recordings

n_rows = len(df)
# make random order of file processing
rec_ids = np.random.choice(np.arange(n_rows), n_rows, replace=False)
print ("rec_ids: ", rec_ids)
#
for rec_id in rec_ids:

    #######################################################################
    ################# FIND FIRST RECORDING NOT PROCESSED ##################
    #######################################################################
    #
    d_ = df['comments'][rec_id]
    if d_==1.0:
        print (" file ", rec_id, " processed, skipping: ")
        continue

    #######################################################################
    #### RESAVE SPREADSHEET - WITH COMPLETE FLAG FOR CURRENT RECORDING  ###
    #######################################################################
    #
    df.iloc[rec_id, df.columns.get_loc('comments')] = 1

    # save corrected excel sheet
    df.to_excel(fname_xls, index=False)

    ###############################################
    ################ GET LABELS ###################
    ###############################################
    #
    mouse_ids = get_mouse_ids(header, df)
    print ("mouse ids: ", mouse_ids)

    #
    paths = get_file_paths(header, df)
    print ("paths: ", paths, paths[0])

    #
    sessions = get_sessions(header, df)
    print ("sessions: ", sessions)

    ##########################################################
    ########## CONVERT .MSEC TO .TIFF AND RUN SUITE2P ########
    ##########################################################

    # get correct file name
    print ("Processing dataset: ", paths[rec_id])
    fname = os.path.join(root_dir,
                         paths[rec_id].replace(r'\\', '/'))
    p = pathlib.PureWindowsPath(fname)
    fname = (p.as_posix())

    # grab session and convert .mesc data to .tiff
    sessions_in = sessions[rec_id]
    if 'mesc' in fname:
        fname_tiff = convert_mesc_sessions_to_concatenated_tiff(fname,
                                                        sessions_in)
    elif 'raw' in fname:
        #print ("root_dir: ", root_dir)
        #print("initial fname: ", fname)
        fname_in = root_dir+ fname
        #print ("fname in: ", fname_in)
        fname_tiff = convert_raw_to_tiff(fname_in, sessions_in)
    #    
    run_suite2p_from_fname(fname_tiff,fname_ops)


rec_ids:  [3 2 0 1]
 file  3  processed, skipping: 
 file  2  processed, skipping: 
 file  0  processed, skipping: 
 file  1  processed, skipping: 


In [ ]:
Users/Catalin/Experiments/DON-010473/20220520/002P-F/DON-010473_20220520_002P-F_S1_S2-ACQ.mesc
Users/Catalin/Experiments/DON-010473/20220520/002P-F/DON-010473_20220520_002P-F_S1-S2-ACQ.mesc

             \Experiments\DON-010473\20220520\002P-F\DON-010473_20220520_002P-F_S1-S2-ACQ.mesc

